In [ ]:

import pandas as pd
import numpy as np
import os
from glob import glob

from tabulate import tabulate
printtab = lambda x : print(tabulate(x, headers='firstrow'))

In [ ]:

base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2022-12-16_15-55-24-478281"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-03-29_14-16-04-931491"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-03-29_14-16-04-931491_updatedSep2023"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-10-11_23-47-30-192455"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-10-24_21-00-37-093241"

base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2024-12-11_07-52-16-945389"

base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2025-07-22_15-08-21-725343"

output_dir = os.path.join(base_dir, "outputs")
outputs_dirs = [os.path.join(output_dir, f"option_{i}") for i in range(6)]

In [ ]:
"""
demo_group:
0 - whole US population
1 - NHB
2 - All Other Ethnicities
3 - Pov
4 - Above Pov
"""
def collect_results(demo_group):
    opt = 0

    outputs = outputs_dirs[opt]
    collection_list = []

    # for each arr, store a 2D array in the list
    # axis = 0 are the groups: menthol, nonmenthol, smoker, menthol/smoker, ecig/dual, former, never (3, 4, 3+4, 3/(3+4), 5, 2, 1)
    # axis = 1 are the years 2016-2066
    for i,f in enumerate(sorted(glob(outputs + "/*.npy"))):
        arr = np.load(f)
        arr = arr[:,:,:,0,:] # age-restrict 18-64
        # don't change any of the years
        # arr = arr[[0, 5, 10, 15, 35]] 
        if demo_group == 0:
            arr = np.sum(arr, axis=(1,2)) # dont care about demographics
        elif demo_group == 1:
            arr = np.sum(arr, axis=2)
            arr = arr[:,1]
        elif demo_group == 2:
            arr = np.sum(arr, axis=2)
            arr = arr[:,0]
        elif demo_group == 3:
            arr = np.sum(arr, axis=1)
            arr = arr[:,1]
        elif demo_group == 4:
            arr = np.sum(arr, axis=1)
            arr = arr[:,0]
        arr = arr[:,:-1] # don't need dead people
        sums = np.sum(arr, axis=1) # total count for each year
        arr = arr / sums[:,np.newaxis] # get proportions
        arr = arr.T # transpose so we have (smoking groups, years) as axes
        arr = np.concatenate([ # want to add the smokers together too
            arr[0:4],
            (arr[2] + arr[3])[np.newaxis, :],
            (arr[2] / (arr[2] + arr[3]))[np.newaxis, :],
            arr[4][np.newaxis, :],
        ], axis=0)
        arr = arr[[2,3,4,5,6,1,0]] # re-order the smoking groups
        # add change 2016-2031 column and change from SQ column
        collection_list.append(arr)

    collection_list = np.array(collection_list)
    return collection_list

    # axis = 0 are the runs done in the uncertainty analysis
    # axis = 1 are the groups: menthol, nonmenthol, smoker, menthol/smoker, ecig/dual, former, never (3, 4, 3+4, 3/(3+4), 5, 2, 1)
    # axis = 2 are the years 2016-2066

        


In [ ]:

def collection_to_meanul(collection_list):

# analyze collection_list and get 95% confidence intervals

    mean_results = np.zeros_like(collection_list[0])
    upper_bound = np.zeros_like(collection_list[0])
    lower_bound = np.zeros_like(collection_list[0])

    for i in range(collection_list.shape[1]):
        for j in range(collection_list.shape[2]):
            mean = np.mean(collection_list[:,i,j])
            upper = np.percentile(collection_list[:,i,j], 97.5)
            lower = np.percentile(collection_list[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower
    
    mean_results = np.round(mean_results, decimals=4) * 100
    upper_bound = np.round(upper_bound, decimals=4) * 100
    lower_bound = np.round(lower_bound, decimals=4) * 100
    mean_results = np.round(mean_results, decimals=2)
    upper_bound = np.round(upper_bound, decimals=2)
    lower_bound = np.round(lower_bound, decimals=2)
    return mean_results, upper_bound, lower_bound


In [ ]:
all_mean, all_u, all_l = collection_to_meanul(collect_results(0))
black_mean, black_u, black_l = collection_to_meanul(collect_results(1))
nonblack_mean, nonblack_u, nonblack_l = collection_to_meanul(collect_results(2))
pov_mean, pov_u, pov_l = collection_to_meanul(collect_results(3))
nonpov_mean, nonpov_u, nonpov_l = collection_to_meanul(collect_results(4))

In [ ]:
print(all_mean.shape)
print(black_mean.shape)

In [ ]:
#prevalence of menthol cigarette use among smokers

print(all_mean[3,:5])
print(all_u[3,:5])
print(all_l[3,:5])

In [ ]:

print(black_mean[3,:5])
print(black_l[3,:5])
print(black_u[3,:5])

In [ ]:

print(pov_mean[3,:5])
print(pov_l[3,:5])
print(pov_u[3,:5])

In [ ]:
# estimates of tobacco use
def pretty_state3(arrm, arrl, arru, start_year=2016, state=3):
    ret = ""
    assert(arrm.shape == arrl.shape)
    assert(arrm.shape == arru.shape)
    datastring = ""
    yearstring = ""
    current_year = start_year
    for year in range(len(arrm[state])):
        thisdata = f"{arrm[state,year]} ({arrl[state,year]}, {arru[state,year]})" + "  "
        datastring += thisdata
        thisyear = str(current_year)
        thisyear += " " * (len(thisdata) - len(thisyear))
        yearstring += thisyear
        current_year += 1
    print(yearstring)
    print(datastring)

In [ ]:
print(all_mean.shape)

In [ ]:

"""
This data is for the paper (table 2)
"""
print("Menthol Smoking Among Cigarette Smokers")
print("All Demographics")
pretty_state3(
    all_mean[:,:5],
    all_l[:,:5],
    all_u[:,:5],
)
print("Non-Hispanic Black")

pretty_state3(
    black_mean[:,:5],
    black_l[:,:5],
    black_u[:,:5],
)
print("Living in Poverty")
pretty_state3(
    pov_mean[:,:5],
    pov_l[:,:5],
    pov_u[:,:5],
)

In [ ]:
# For Table 3
def pretty(arrm, arrl, arru, years=[2020,2030,2040,2050], states = ["Smoker", "Menthol", "Non-Menthol", "E-cig/Dual"]):
    ret = ""
    assert(arrm.shape == arrl.shape)
    assert(arrm.shape == arru.shape)
    for state in range(len(arrm)):
        statename = states[state]
        datastring = statename + " " * (15 - len(statename))

        yearstring =  " " * len(datastring)
        for year in range(len(arrm[state])):
            thisdata = f"{arrm[state,year]} ({arrl[state,year]}, {arru[state,year]})" + "  "
            thisdata += " " * (25 - len(thisdata))
            datastring += thisdata

            thisyear = str(years[year])
            thisyear += " " * (len(thisdata) - len(thisyear))
            yearstring += thisyear
        if (state==0): print(yearstring)
        print(datastring)

In [ ]:

print("ALL DEMOGRAPHICS")
pretty(
    all_mean[[2,0,1,4]][:,[4,14,24,34]],
    all_l[[2,0,1,4]][:,[4,14,24,34]],
    all_u[[2,0,1,4]][:,[4,14,24,34]],
)

print("")
print("Non-Hispanic Black")
pretty(
    black_mean[[2,0,1,4]][:,[4,14,24,34]],
    black_l[[2,0,1,4]][:,[4,14,24,34]],
    black_u[[2,0,1,4]][:,[4,14,24,34]],
)

print("")
print("All Other Ethnicities")
pretty(
    nonblack_mean[[2,0,1,4]][:,[4,14,24,34]],
    nonblack_l[[2,0,1,4]][:,[4,14,24,34]],
    nonblack_u[[2,0,1,4]][:,[4,14,24,34]],
)

print("")
print("Living in Poverty")
pretty(
    pov_mean[[2,0,1,4]][:,[4,14,24,34]],
    pov_l[[2,0,1,4]][:,[4,14,24,34]],
    pov_u[[2,0,1,4]][:,[4,14,24,34]],
)

print("")
print("Above Poverty Line")
pretty(
    nonpov_mean[[2,0,1,4]][:,[4,14,24,34]],
    nonpov_l[[2,0,1,4]][:,[4,14,24,34]],
    nonpov_u[[2,0,1,4]][:,[4,14,24,34]],
)

In [ ]:

# For Table 5
def pretty(arrm, arrl, arru, years=[2016,2017,2018,2019,2020,2021], states = ["Smoker", "E-cig/Dual"]):
    ret = ""
    assert(arrm.shape == arrl.shape)
    assert(arrm.shape == arru.shape)
    for state in range(len(arrm)):
        statename = states[state]
        datastring = statename + " " * (15 - len(statename))

        yearstring =  " " * len(datastring)
        for year in range(len(arrm[state])):
            thisdata = f"{arrm[state,year]} ({arrl[state,year]}, {arru[state,year]})" + "  "
            thisdata += " " * (25 - len(thisdata))
            datastring += thisdata

            thisyear = str(years[year])
            thisyear += " " * (len(thisdata) - len(thisyear))
            yearstring += thisyear
        if (state==0): print(yearstring)
        print(datastring)

In [ ]:
print(black_mean.shape)
print(black_l.shape)
print(black_u.shape)

In [ ]:

# Table 5
print("ALL DEMOGRAPHICS")
pretty(
    all_mean[[2,4]][:,:6],
    all_l[[2,4]][:,:6],
    all_u[[2,4]][:,:6],
)

print("")
print("Non-Hispanic Black")
pretty(
    black_mean[[2,4]][:,:6],
    black_l[[2,4]][:,:6],
    black_u[[2,4]][:,:6],
)

print("")
print("All Other Ethnicities")
pretty(
    nonblack_mean[[2,4]][:,:6],
    nonblack_l[[2,4]][:,:6],
    nonblack_u[[2,4]][:,:6],
)

print("")
print("Living in Poverty")
pretty(
    pov_mean[[2,4]][:,:6],
    pov_l[[2,4]][:,:6],
    pov_u[[2,4]][:,:6],
)

print("")
print("Above Poverty Line")
pretty(
    nonpov_mean[[2,4]][:,:6],
    nonpov_l[[2,4]][:,:6],
    nonpov_u[[2,4]][:,:6],
)